In [1]:
## Data Preparation Library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

##Model Library
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.layers import Dropout,ELU
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.utils import to_categorical
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from keras.layers import Activation, Dense, BatchNormalization, Dropout
from keras_radam import RAdam

Using TensorFlow backend.


## Reading the dataset:

In [2]:
mf_train  = pd.read_csv("Train.csv")
mf_test = pd.read_csv("Test.csv")
sb = pd.read_excel('Sample_Submission.xlsx')

In [3]:
mf_train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f19,f20,f21,f22,f23,f24,f25,f26,f27,grade
0,1.848564,-0.26425,-0.461423,0.409400,1.305455,2.329398,0.370965,0.090167,0.107958,0.0,...,0.085505,0.233285,-1.080663,0.443257,-0.406121,-0.687687,0.271886,3.727218,0.102129,2
1,-0.825098,-0.26425,3.032397,-2.442599,1.305455,-0.276144,0.370965,0.090167,0.107958,0.0,...,0.085505,0.233285,-1.080663,-0.232546,-0.406366,-0.687687,0.271886,-0.232472,0.102129,4
2,1.848564,-0.26425,-0.461423,0.409400,1.305455,2.329398,0.370965,0.090167,0.107958,0.0,...,0.085505,0.233285,0.925358,1.459782,1.221876,1.877777,0.271886,-0.232472,0.102129,2
3,0.511733,-0.26425,-0.461423,0.409400,-0.525726,-0.276144,0.370965,0.090167,0.107958,0.0,...,0.085505,0.233285,0.925358,-0.008030,-0.406366,1.504523,0.271886,-0.232472,0.102129,2
4,-0.825098,-0.26425,-0.461423,0.409400,-0.525726,-0.276144,0.370965,0.090167,0.107958,0.0,...,0.085505,0.233285,0.925358,-0.573268,-1.164793,1.877777,0.271886,-0.232472,0.102129,2


In [4]:
mf_train['grade'].unique()

array([2, 4, 3, 1, 0], dtype=int64)

In [5]:
mf_test.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27
0,-0.837812,-0.273636,1.276580,0.463262,-0.585142,-0.24287,0.349804,0.12356,0.166795,0.06143,...,0.197642,0.06143,0.27735,0.886135,-0.568935,1.100428,-0.244589,0.229718,-0.217109,0.087039
1,2.078087,-0.273636,-0.496119,0.463262,-2.438092,-0.24287,0.349804,0.12356,0.166795,0.06143,...,-5.059644,0.06143,0.27735,0.886135,0.504299,-0.434268,-0.244040,0.229718,-0.217109,0.087039
2,-0.837812,-0.273636,1.276580,0.463262,-0.585142,-0.24287,0.349804,0.12356,0.166795,0.06143,...,0.197642,0.06143,0.27735,-1.128496,-0.568935,-0.434268,-0.662763,0.229718,-0.217109,0.087039
3,-0.837812,-0.273636,-0.496119,0.463262,1.267808,-0.24287,-2.858743,0.12356,0.166795,0.06143,...,-5.059644,0.06143,0.27735,-1.128496,-0.449819,-1.918647,-0.662763,0.229718,-0.217109,0.087039
4,-0.837812,-0.273636,-0.496119,0.463262,-0.585142,-0.24287,-2.858743,0.12356,0.166795,0.06143,...,0.197642,0.06143,0.27735,-1.128496,-0.568935,-0.434268,-0.662763,0.229718,-0.217109,0.087039


In [6]:
sb.head()

,0,1,2,3,4
0,0,0,1,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,1,0,0


In [7]:
sb.shape

(266, 5)

In [8]:
x_train = mf_train.drop(columns=['grade'])

In [9]:
y_train = mf_train['grade']

In [10]:
x_test = mf_test

In [11]:
x_test.shape

(266, 28)

In [12]:
y_train.shape

(620,)

In [13]:
x_train.shape

(620, 28)

In [14]:
std = StandardScaler()
x_std = std.fit_transform(x_train)

In [15]:
x_std.shape[1]

28

### Training & Validating Model
- Measures to improve training is applied simultaneously
- More training set
- Weight Initialization scheme
- Available activations (elu)
  - Fast and Accurate Deep Network Learning by Exponential Linear Units (ELUs)
- Optimizers: adaptvie
- Batch Normalization
- Dropout (Regularization)
- Model Ensemble

In [16]:
def neural_network():
    classifier = Sequential()

     # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 100, kernel_initializer = 'he_normal', activation = 'elu', input_dim = x_std.shape[1]))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(rate = 0.1))
    
    # Adding the second hidden layer
    classifier.add(Dense(units = 100, kernel_initializer = 'he_normal', activation = 'elu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(rate = 0.1))
    
     # Adding the third hidden layer
    classifier.add(Dense(units = 100, kernel_initializer = 'he_normal', activation = 'elu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(rate = 0.1))
    
     # Adding the four hidden layer
    classifier.add(Dense(units = 100, kernel_initializer = 'he_normal', activation = 'elu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(rate = 0.1))
    

    classifier.add(Dense(5,activation='softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    classifier.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])   
    ##classifier.compile(RAdam(), loss='categorical_crossentropy', metrics=['acc'])   
    return classifier;

In [17]:
# create 5 models to ensemble
model1 = KerasClassifier(build_fn = neural_network, epochs = 100)
model2 = KerasClassifier(build_fn = neural_network, epochs = 100)
model3 = KerasClassifier(build_fn = neural_network, epochs = 100)
model4 = KerasClassifier(build_fn = neural_network, epochs = 100)
model5 = KerasClassifier(build_fn = neural_network, epochs = 100)

In [18]:
ensemble_clf = VotingClassifier(estimators = [('model1', model1), ('model2', model2), ('model3', model3), ('model4', model4), ('model5', model5)], voting = 'soft')

In [19]:
ensemble_clf

VotingClassifier(estimators=[('model1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFD964C8>),
                             ('model2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFDC7608>),
                             ('model3',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFD96588>),
                             ('model4',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFD96D48>),
                             ('model5',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFD966C8>)],
                 flatten_transform=True, n_jobs=None, voting='soft',
                 weights=None)

## For Ensemble Approach, Target value has to be reshape in the form numpy.ndarray

In [20]:
y_train=mf_train[['grade']].values
y_train = y_train.reshape(len(y_train))

In [21]:
y_train

array([2, 4, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 2, 3, 3, 1, 3, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 1,
       1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       1, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 4, 0, 2, 1, 1,
       2, 2, 1, 4, 2, 2, 2, 3, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 3, 1, 2, 4, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2,
       2, 1, 2, 2, 2, 1, 2, 2, 2, 3, 2, 1, 1, 4, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 1, 1, 2, 3, 4, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 4, 2, 2, 2,
       2, 4, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2,

In [22]:
type(y_train)

numpy.ndarray

In [23]:
ensemble_clf.fit(x_std, y_train)

Epoch 1/100
620/620 [==============================] - 6s 10ms/step - loss: 1.7521 - acc: 0.3306
Epoch 2/100
620/620 [==============================] - 0s 325us/step - loss: 1.1872 - acc: 0.5581
Epoch 3/100
620/620 [==============================] - ETA: 0s - loss: 1.0112 - acc: 0.610 - 0s 284us/step - loss: 0.9785 - acc: 0.6323
Epoch 4/100
620/620 [==============================] - 0s 338us/step - loss: 0.7611 - acc: 0.7468
Epoch 5/100
620/620 [==============================] - 0s 297us/step - loss: 0.6857 - acc: 0.7935
Epoch 6/100
620/620 [==============================] - 0s 319us/step - loss: 0.5750 - acc: 0.8081
Epoch 7/100
620/620 [==============================] - 0s 322us/step - loss: 0.4890 - acc: 0.8371
Epoch 8/100
620/620 [==============================] - 0s 313us/step - loss: 0.4771 - acc: 0.8484
Epoch 9/100
620/620 [==============================] - 0s 283us/step - loss: 0.4415 - acc: 0.8516
Epoch 10/100
620/620 [==============================] - 0s 300us/step - loss: 0.3

Epoch 82/100
620/620 [==============================] - 0s 317us/step - loss: 0.1235 - acc: 0.9565 0s - loss: 0.1245 - acc: 0.95
Epoch 83/100
620/620 [==============================] - 0s 290us/step - loss: 0.1321 - acc: 0.9548
Epoch 84/100
620/620 [==============================] - 0s 357us/step - loss: 0.1144 - acc: 0.9629
Epoch 85/100
620/620 [==============================] - 0s 281us/step - loss: 0.1388 - acc: 0.9339
Epoch 86/100
620/620 [==============================] - 0s 312us/step - loss: 0.1445 - acc: 0.9468
Epoch 87/100
620/620 [==============================] - 0s 292us/step - loss: 0.1140 - acc: 0.9613
Epoch 88/100
620/620 [==============================] - 0s 315us/step - loss: 0.1110 - acc: 0.9597
Epoch 89/100
620/620 [==============================] - 0s 275us/step - loss: 0.1278 - acc: 0.9500
Epoch 90/100
620/620 [==============================] - 0s 307us/step - loss: 0.1089 - acc: 0.9613
Epoch 91/100
620/620 [==============================] - 0s 266us/step - loss: 0

620/620 [==============================] - 0s 280us/step - loss: 0.1276 - acc: 0.9516
Epoch 64/100
620/620 [==============================] - 0s 274us/step - loss: 0.1488 - acc: 0.9500
Epoch 65/100
620/620 [==============================] - 0s 276us/step - loss: 0.1387 - acc: 0.9435
Epoch 66/100
620/620 [==============================] - 0s 330us/step - loss: 0.1516 - acc: 0.9274
Epoch 67/100
620/620 [==============================] - 0s 337us/step - loss: 0.1482 - acc: 0.9419
Epoch 68/100
620/620 [==============================] - 0s 280us/step - loss: 0.1338 - acc: 0.9532
Epoch 69/100
620/620 [==============================] - 0s 298us/step - loss: 0.1335 - acc: 0.9500
Epoch 70/100
620/620 [==============================] - 0s 266us/step - loss: 0.1284 - acc: 0.9516
Epoch 71/100
620/620 [==============================] - 0s 288us/step - loss: 0.1540 - acc: 0.9435
Epoch 72/100
620/620 [==============================] - 0s 271us/step - loss: 0.1375 - acc: 0.9419
Epoch 73/100
620/620 [=

620/620 [==============================] - 0s 293us/step - loss: 0.1571 - acc: 0.9387
Epoch 46/100
620/620 [==============================] - 0s 267us/step - loss: 0.1718 - acc: 0.9387
Epoch 47/100
620/620 [==============================] - 0s 302us/step - loss: 0.1597 - acc: 0.9435
Epoch 48/100
620/620 [==============================] - 0s 286us/step - loss: 0.1665 - acc: 0.9306
Epoch 49/100
620/620 [==============================] - 0s 291us/step - loss: 0.1418 - acc: 0.9387
Epoch 50/100
620/620 [==============================] - 0s 271us/step - loss: 0.1742 - acc: 0.9355
Epoch 51/100
620/620 [==============================] - 0s 298us/step - loss: 0.1696 - acc: 0.9387
Epoch 52/100
620/620 [==============================] - 0s 284us/step - loss: 0.1445 - acc: 0.9323
Epoch 53/100
620/620 [==============================] - 0s 295us/step - loss: 0.1552 - acc: 0.9355
Epoch 54/100
620/620 [==============================] - 0s 289us/step - loss: 0.1513 - acc: 0.9339
Epoch 55/100
620/620 [=

620/620 [==============================] - 0s 355us/step - loss: 0.1979 - acc: 0.9145
Epoch 28/100
620/620 [==============================] - 0s 366us/step - loss: 0.2097 - acc: 0.9113
Epoch 29/100
620/620 [==============================] - 0s 327us/step - loss: 0.1965 - acc: 0.9226
Epoch 30/100
620/620 [==============================] - 0s 306us/step - loss: 0.1809 - acc: 0.9435
Epoch 31/100
620/620 [==============================] - 0s 427us/step - loss: 0.1789 - acc: 0.9290
Epoch 32/100
620/620 [==============================] - 0s 310us/step - loss: 0.1994 - acc: 0.9323
Epoch 33/100
620/620 [==============================] - 0s 335us/step - loss: 0.1864 - acc: 0.9339
Epoch 34/100
620/620 [==============================] - 0s 315us/step - loss: 0.1913 - acc: 0.9177
Epoch 35/100
620/620 [==============================] - 0s 330us/step - loss: 0.1777 - acc: 0.9161
Epoch 36/100
620/620 [==============================] - 0s 310us/step - loss: 0.1446 - acc: 0.9452 0s - loss: 0.1461 - acc

620/620 [==============================] - 0s 301us/step - loss: 0.4279 - acc: 0.8548
Epoch 10/100
620/620 [==============================] - 0s 271us/step - loss: 0.4004 - acc: 0.8565
Epoch 11/100
620/620 [==============================] - 0s 287us/step - loss: 0.3806 - acc: 0.8726
Epoch 12/100
620/620 [==============================] - 0s 300us/step - loss: 0.3609 - acc: 0.8774
Epoch 13/100
620/620 [==============================] - 0s 303us/step - loss: 0.3281 - acc: 0.8968
Epoch 14/100
620/620 [==============================] - 0s 305us/step - loss: 0.2824 - acc: 0.9032
Epoch 15/100
620/620 [==============================] - 0s 285us/step - loss: 0.3112 - acc: 0.8887
Epoch 16/100
620/620 [==============================] - 0s 298us/step - loss: 0.2921 - acc: 0.8984 0s - loss: 0.2970 - acc: 0.90
Epoch 17/100
620/620 [==============================] - 0s 274us/step - loss: 0.2991 - acc: 0.8871
Epoch 18/100
620/620 [==============================] - 0s 277us/step - loss: 0.2661 - acc: 

Epoch 91/100
620/620 [==============================] - 0s 283us/step - loss: 0.1172 - acc: 0.9629
Epoch 92/100
620/620 [==============================] - 0s 271us/step - loss: 0.1343 - acc: 0.9516
Epoch 93/100
620/620 [==============================] - 0s 296us/step - loss: 0.1501 - acc: 0.9403
Epoch 94/100
620/620 [==============================] - 0s 292us/step - loss: 0.1258 - acc: 0.9532
Epoch 95/100
620/620 [==============================] - 0s 297us/step - loss: 0.1218 - acc: 0.9468
Epoch 96/100
620/620 [==============================] - 0s 289us/step - loss: 0.1140 - acc: 0.9613
Epoch 97/100
620/620 [==============================] - 0s 276us/step - loss: 0.1138 - acc: 0.9597
Epoch 98/100
620/620 [==============================] - 0s 276us/step - loss: 0.1234 - acc: 0.9468
Epoch 99/100
620/620 [==============================] - 0s 304us/step - loss: 0.1346 - acc: 0.9565
Epoch 100/100
620/620 [==============================] - 0s 289us/step - loss: 0.1072 - acc: 0.9581


VotingClassifier(estimators=[('model1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFD964C8>),
                             ('model2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFDC7608>),
                             ('model3',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFD96588>),
                             ('model4',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFD96D48>),
                             ('model5',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BFD966C8>)],
                 flatten_transform=True, n_jobs=None, voting='soft',
                 weights=None)

In [24]:
ytrain_predicted = ensemble_clf.predict(x_train)

In [25]:
print('Acc: ', accuracy_score(ytrain_predicted, y_train))

Acc:  0.9790322580645161


#### Final model produces near 98% of accuracy, which is rather improved from before

In [26]:
ytest_predicted = ensemble_clf.predict(x_test)

In [27]:
ytest_predicted

array([2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2,
       2, 2, 4, 2, 3, 3, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
       2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 3, 2, 2, 2,
       2, 2, 2, 2, 0, 4, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 4, 2, 2, 3, 1, 2, 1, 2, 3,
       2, 2, 2, 2, 2, 3, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 2, 2, 1, 4, 2, 3,
       2, 2, 2, 2, 1, 1, 2, 1, 2, 3, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 3, 2, 4, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 3, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 3, 2, 2, 1, 1,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

In [28]:
sub_array=to_categorical(ytest_predicted).astype(int)
result=pd.DataFrame(data=sub_array,  columns=[0, 1,2,3,4])
result.to_excel('adam.xlsx',index=False)

In [29]:
result.head()

,0,1,2,3,4
0,0,0,1,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,1,0,0
